In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/.shortcut-targets-by-id/1VhaW8PqOm10rxodZbXmRV2GHxBJq9Qxh/Data Open Championship 2022/2-data/preprocessed'

Mounted at /content/drive


In [ ]:
forpro_df = pd.read_csv(path + '/forpro_inst_fin_cleaned.csv').drop(columns=['Unnamed: 0'])
nonpro_df = pd.read_csv(path + '/nonpro_inst_fin_cleaned.csv').drop(columns=['Unnamed: 0'])
public_df = pd.read_csv(path + '/pub_inst_fin_cleaned.csv').drop(columns=['Unnamed: 0'])
aid_df = pd.read_csv(path + '/finaid.csv').drop(columns=['Unnamed: 0'])
comp_df = pd.read_csv(path + '/df_completions_final.csv').drop(columns=['Unnamed: 0'])
grad_df = pd.read_csv(path + '/df_gradrate1.csv')
inst_df = pd.read_csv(path + '/df_institutional_master.csv').drop(columns=['Unnamed: 0'])
df_lst = [forpro_df, nonpro_df, public_df, aid_df, comp_df, grad_df, inst_df]

inst_df.loc[inst_df.housing_roomcapacity == ".", 'housing_roomcapacity'] = "0"
inst_df['housing_roomcapacity'].astype(np.float)

id_lst = list(inst_df.unitid)

for table in df_lst:
  table = table[table.unitid.isin(id_lst)]

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (130) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (177) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
<ipython-input-4-e15772409d2b>:11: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  inst_df['housing_roomcapacity'].astype(np.float)


In [ ]:
#public_df = public_df.rename(columns = {'endyear':'year'})
public_df['year'] = 2000 + public_df['endyear']
public_df

In [ ]:
list(grad_df.columns)[:17]

grad_df_total = grad_df.iloc[:, :17]
grad_df_total

In [ ]:
id_not4year = set(grad_df_total[grad_df_total.gradrate_4yr_total_.isna()].unitid)

df_not4year = inst_df[inst_df.unitid.isin(id_not4year)]

In [ ]:
inst_df.iloc[:, :18].head()

Impact of costs on outcome measures

In [ ]:
# for profit, non profit, public
# 4 year, 2 year, other
# aid + finance

currdf = grad_df.merge(inst_df, on=['unitid'], how='right') #why is gradrate 4 year here?
currdf

In [ ]:
#set(inst_df.ugoffer), {'No undergraduate offering', 'Undergraduate degree or certificate offering'}
#set(inst_df.iclevel), {'At least 2 but less than 4 years', 'Four or more years'}
#for public

public = currdf.merge(public_df, on = ['unitid', 'year'], how = 'inner')
public = public[(public.ugoffer == 'Undergraduate degree or certificate offering') & (public.iclevel == 'At least 2 but less than 4 years') & (public.gradrate_2yr_total_.notna())].dropna(axis=1, how='all')

In [ ]:
public

In [19]:
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

y_lst = ['gradrate_2yr_total_',
 'gradrate_2yr_completer150_',
 'gradrate_2yr_transfer_',
 'gradrate_2yr_nocompleteren_',
 'gradrate_2yr_nocompleter_',
 'gradrate_2yr_total_men']

X = public.iloc[:,50:] #.drop(columns = ['unitid', 'endyear'] + y_lst)
y = public.gradrate_2yr_completer150_

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

#for i in range(10):
  #model = sm.OLS.fit_regularized(public.gradrate_2yr_completer150_, public.speclearning_distancecourse, alpha = 0, L1_wt = 1)
  #results = model.fit
  #print(results.t_test([1, 0]))

#results = model.fit()
obj =  ['housing_roomcapacity', 'pension information reported', 'does this institution or any of its foundations or other affiliated organizations own endowment assets ?','postemployment benefits other than pension (opeb) reported']

X = X.select_dtypes(exclude=['object'])

NameError: ignored

In [ ]:
X = X[['tuition and fees, after deducting discounts and allowances',
 'federal operating grants and contracts', 'state operating grants and contracts', 'local/private operating grants and contracts', 'local operating grants and contracts','federal appropriations',
 'state appropriations',
 'local appropriations, education district taxes, and similar support',
 'federal nonoperating grants',
 'state nonoperating grants',
 'local nonoperating grants',
 'gifts, including contributions from affiliated organizations',
 'investment income',
 'other nonoperating revenues',
 'total nonoperating revenues',
 'total operating and nonoperating revenues',
 'capital appropriations',
 'capital grants and gifts',
 'additions to permanent endowments',
 'other revenues and additions',
 'total other revenues and additions',
 'total all revenues and other additions',
 'instruction - current year total',
 'instruction - salaries and wages',
 'research - current year total',
 'research - salaries and wages',
 'public service - current year total',
 'public service - salaries and wages',
 'academic support - current year total',
 'academic support - salaries and wages',
 'student services - current year total',
 'student services - salaries and wages',
 'institutional support - current year total',
 'institutional support - salaries and wages',
'pell grants (federal)',
 'other federal grants',
 'grants by state government',
 'grants by local government',
 'institutional grants from restricted resources',
 'institutional grants from unrestricted resources',
 'total gross scholarships and fellowships',
 'discounts and allowances applied to tuition and fees',
 'discounts and allowances applied to sales & services of auxiliary enterprises',
 'total discounts and allowances']]
X.info(verbose = True, show_counts=True)

In [ ]:
covars = ['total current assets', 'long-term debt', 'total liabilities', 'total net assets', 'tuition and fees, after deducting discounts and allowances',
 'federal operating grants and contracts', 'state operating grants and contracts', 'local/private operating grants and contracts', 'local operating grants and contracts','federal appropriations',
 'state appropriations',
 'local appropriations, education district taxes, and similar support',
 'federal nonoperating grants',
 'state nonoperating grants',
 'local nonoperating grants',
 'gifts, including contributions from affiliated organizations',
 'investment income',
 'other nonoperating revenues',
 'total nonoperating revenues',
 'total operating and nonoperating revenues',
 'capital appropriations',
 'capital grants and gifts',
 'additions to permanent endowments',
 'other revenues and additions',
 'total other revenues and additions',
 'total all revenues and other additions',
 'instruction - current year total',
 'instruction - salaries and wages',
 'research - current year total',
 'research - salaries and wages',
 'public service - current year total',
 'public service - salaries and wages',
 'academic support - current year total',
 'academic support - salaries and wages',
 'student services - current year total',
 'student services - salaries and wages',
 'institutional support - current year total',
 'institutional support - salaries and wages',
'pell grants (federal)',
 'other federal grants',
 'grants by state government',
 'grants by local government',
 'institutional grants from restricted resources',
 'institutional grants from unrestricted resources',
 'total gross scholarships and fellowships',
 'discounts and allowances applied to tuition and fees',
 'discounts and allowances applied to sales & services of auxiliary enterprises',
 'total discounts and allowances']

from sklearn import linear_model

for i in range(11):
  clf = linear_model.Lasso(alpha=i*10)
  model = clf.fit(X, y)
  print(model.intercept_, model.coef_, model.score(X, y))

In [3]:
forpror = pd.read_csv(path + '/relative/df_all_forpro.csv').drop(columns=['Unnamed: 0'])
forproa = pd.read_csv(path + '/absolute/df_all_forpro.csv').drop(columns=['Unnamed: 0'])

nonpror = pd.read_csv(path + '/relative/df_all_nonpro.csv').drop(columns=['Unnamed: 0'])
nonproa = pd.read_csv(path + '/absolute/df_all_nonpro.csv').drop(columns=['Unnamed: 0'])

pubr = pd.read_csv(path + '/relative/df_all_pub.csv').drop(columns=['Unnamed: 0'])
puba = pd.read_csv(path + '/absolute/df_all_pub.csv').drop(columns=['Unnamed: 0'])

In [4]:
def get_pvals(relative_df, grad_df, n, k):
  lst = []
  for i in range(k + 1):
    lst_i = []
    curr = relative_df[relative_df.columns[i::k]]
    if len(list(curr)) == 0:
      continue
    lst_i.append(list(curr)[0])
    curr_y = grad_df[grad_df.columns[1 + 5*n::105]]
    Y = curr_y.mean(axis = 1)
    lst_i.append(list(curr_y)[0])
    curr_name = list(curr)[0]
    X = curr.mean(axis = 1)
    curr = pd.concat([X, Y], axis = 1).dropna()
    X = curr.iloc[:, 0]
    Y = curr.iloc[:, 1]
    if len(X) == 0 or len(Y) == 0:
      continue
    X = sm.add_constant(X)
    model = sm.OLS(Y,X)
    if len(list(model.fit().params)) < 2:
      continue
    lst_i.append(list(model.fit().params)[1])
    lst_i.append(list(model.fit().pvalues)[1])
    lst.append(lst_i)
  return pd.DataFrame(lst, columns = ['X', 'Y', 'coef', 'p_val'])

In [ ]:
def get_pvals(relative_df, grad_df, n, k):
  lst = []
  for i in range(k + 1):
    list_i = []
    curr = relative_df[relative_df.columns[i::k]]

In [ ]:
aid = forpror[forpror.columns[forpror.columns.str.startswith("finaid")]] #finaid (27)  (1415, 1516, 1617, 1718, 1819, 1920), finance, gradrate (105)
finance = forpror[forpror.columns[forpror.columns.str.startswith("finance")]]
grad = forpror[forpror.columns[forpror.columns.str.startswith("gradrate")]]
fee = forpror[forpror.columns[forpror.columns.str.startswith("fee")]]

In [ ]:
def get_all(main_df, name):
  aid = main_df[main_df.columns[main_df.columns.str.startswith("finaid")]]
  finance = main_df[main_df.columns[main_df.columns.str.startswith("finance")]]
  grad = main_df[main_df.columns[main_df.columns.str.startswith("gradrate")]]
  fee = main_df[main_df.columns[main_df.columns.str.startswith("fee")]]
  lst_df = []
  for i in range(22):
    lst_df.append(get_pvals(aid, grad, i, 27))
  pd.concat(lst_df).to_csv('/content/drive/.shortcut-targets-by-id/1VhaW8PqOm10rxodZbXmRV2GHxBJq9Qxh/Data Open Championship 2022/2-data/preprocessed/aid_grad_' + name + '.csv')
  lst_df = []
  for i in range(22):
    lst_df.append(get_pvals(finance, grad, i, 226))
  pd.concat(lst_df).to_csv('/content/drive/.shortcut-targets-by-id/1VhaW8PqOm10rxodZbXmRV2GHxBJq9Qxh/Data Open Championship 2022/2-data/preprocessed/finance_grad_' + name + '.csv')
  lst_df = []
  for i in range(22):
    lst_df.append(get_pvals(fee, grad, i, 25))
  pd.concat(lst_df).to_csv('/content/drive/.shortcut-targets-by-id/1VhaW8PqOm10rxodZbXmRV2GHxBJq9Qxh/Data Open Championship 2022/2-data/preprocessed/fee_grad_' + name + '.csv')

get_all(forpror, 'forpro')
get_all(pubr, 'pub')
get_all(nonpror, 'nonpro')

In [68]:
import statsmodels.stats.multitest as smt

In [ ]:
df = pd.read_csv('/content/drive/.shortcut-targets-by-id/1VhaW8PqOm10rxodZbXmRV2GHxBJq9Qxh/Data Open Championship 2022/2-data/preprocessed/finance_grad_forpro.csv')
print(df.shape)
df = df.sort_values(by=['p_val']).dropna().drop(columns=['Unnamed: 0'])
agforpro = df[df.p_val < 0.05]
agforpro

In [ ]:
a = agforpro.groupby(['Y']).count().reset_index()
agforpro

In [ ]:
df = pd.read_csv('/content/drive/.shortcut-targets-by-id/1VhaW8PqOm10rxodZbXmRV2GHxBJq9Qxh/Data Open Championship 2022/2-data/preprocessed/aid_grad_forpro.csv')
df.X = df.X.map(lambda x: x.rstrip('1234567890'))
df.Y = df.Y.map(lambda x: x.rstrip('1234567890'))
print(df.shape)
df = df.sort_values(by=['p_val']).dropna().drop(columns=['Unnamed: 0'])
agforpro = df[df.p_val < 0.05]
corrected = df[smt.multipletests(df.p_val, method = 'holm')[0]]

a = agforpro.groupby(['X']).count().reset_index()
plt.pie(a.Y, labels = a.X)
plt.show()

a = agforpro.groupby(['Y']).count().reset_index()
plt.pie(a.X, labels = a.Y)
plt.show() 

a = corrected.groupby(['X']).count().reset_index()
plt.pie(a.Y, labels = a.X)
plt.show()

a = corrected.groupby(['Y']).count().reset_index()
plt.pie(a.X, labels = a.Y)
plt.show()
corrected

In [ ]:
df = pd.read_csv('/content/drive/.shortcut-targets-by-id/1VhaW8PqOm10rxodZbXmRV2GHxBJq9Qxh/Data Open Championship 2022/2-data/preprocessed/aid_grad_nonpro.csv')
df.X = df.X.map(lambda x: x.rstrip('1234567890'))
df.Y = df.Y.map(lambda x: x.rstrip('1234567890'))
print(df.shape)
df = df.sort_values(by=['p_val']).dropna().drop(columns=['Unnamed: 0'])
full = df[df.p_val < 0.05]
corrected = df[smt.multipletests(df.p_val, method = 'holm')[0]]

a = full.groupby(['X']).count().reset_index()
plt.pie(a.Y, labels = a.X)
plt.show()

a = full.groupby(['Y']).count().reset_index()
plt.pie(a.X, labels = a.Y)
plt.show() 

a = corrected.groupby(['X']).count().reset_index()
plt.pie(a.Y, labels = a.X)
plt.show()

a = corrected.groupby(['Y']).count().reset_index()
plt.pie(a.X, labels = a.Y)
plt.show()
corrected

In [ ]:
df = pd.read_csv('/content/drive/.shortcut-targets-by-id/1VhaW8PqOm10rxodZbXmRV2GHxBJq9Qxh/Data Open Championship 2022/2-data/preprocessed/aid_grad_pub.csv')
df.X = df.X.map(lambda x: x.rstrip('1234567890'))
df.Y = df.Y.map(lambda x: x.rstrip('1234567890'))
print(df.shape)
df = df.sort_values(by=['p_val']).dropna().drop(columns=['Unnamed: 0'])
full = df[df.p_val < 0.05]
corrected = df[smt.multipletests(df.p_val, method = 'holm')[0]]

a = full.groupby(['X']).count().reset_index()
plt.pie(a.Y, labels = a.X)
plt.show()

a = full.groupby(['Y']).count().reset_index()
plt.pie(a.X, labels = a.Y)
plt.show() 

a = corrected.groupby(['X']).count().reset_index()
plt.pie(a.Y, labels = a.X)
plt.show()

a = corrected.groupby(['Y']).count().reset_index()
plt.pie(a.X, labels = a.Y)
plt.show()
corrected

In [ ]:
df = pd.read_csv('/content/drive/.shortcut-targets-by-id/1VhaW8PqOm10rxodZbXmRV2GHxBJq9Qxh/Data Open Championship 2022/2-data/preprocessed/finance_grad_forpro.csv')
df.X = df.X.map(lambda x: x.rstrip('1234567890'))
df.Y = df.Y.map(lambda x: x.rstrip('1234567890'))
print(df.shape)
df = df.sort_values(by=['p_val']).dropna().drop(columns=['Unnamed: 0'])
full = df[df.p_val < 0.05]
corrected = df[smt.multipletests(df.p_val, method = 'holm')[0]]

a = full.groupby(['X']).count().reset_index()
plt.pie(a.Y, labels = a.X)
plt.show()

a = full.groupby(['Y']).count().reset_index()
plt.pie(a.X, labels = a.Y)
plt.show() 

a = corrected.groupby(['X']).count().reset_index()
plt.pie(a.Y, labels = a.X)
plt.show()

a = corrected.groupby(['Y']).count().reset_index()
plt.pie(a.X, labels = a.Y)
plt.show()
corrected

In [ ]:
df = pd.read_csv('/content/drive/.shortcut-targets-by-id/1VhaW8PqOm10rxodZbXmRV2GHxBJq9Qxh/Data Open Championship 2022/2-data/preprocessed/finance_grad_nonpro.csv')
df.X = df.X.map(lambda x: x.rstrip('1234567890'))
df.Y = df.Y.map(lambda x: x.rstrip('1234567890'))
print(df.shape)
df = df.sort_values(by=['p_val']).dropna().drop(columns=['Unnamed: 0'])
full = df[df.p_val < 0.05]
corrected = df[smt.multipletests(df.p_val, method = 'holm')[0]]

a = full.groupby(['X']).count().reset_index()
plt.pie(a.Y, labels = a.X)
plt.show()

a = full.groupby(['Y']).count().reset_index()
plt.pie(a.X, labels = a.Y)
plt.show() 

a = corrected.groupby(['X']).count().reset_index()
plt.pie(a.Y, labels = a.X)
plt.show()

a = corrected.groupby(['Y']).count().reset_index()
plt.pie(a.X, labels = a.Y)
plt.show()
corrected
# Notes - 4 year institutions greatly benefit from gifts and thus also their ability to spend the gifts on the university

In [ ]:
 finance[finance.columns[finance.columns.str.startswith("finance_auxiliary enterprises-total amount")]].plot.hist()

In [ ]:
df = pd.read_csv('/content/drive/.shortcut-targets-by-id/1VhaW8PqOm10rxodZbXmRV2GHxBJq9Qxh/Data Open Championship 2022/2-data/preprocessed/finance_grad_pub.csv')
df.X = df.X.map(lambda x: x.rstrip('1234567890'))
df.Y = df.Y.map(lambda x: x.rstrip('1234567890'))
print(df.shape)
df = df.sort_values(by=['p_val']).dropna().drop(columns=['Unnamed: 0'])
full = df[df.p_val < 0.05]
corrected = df[smt.multipletests(df.p_val, method = 'holm')[0]]

a = full.groupby(['X']).count().reset_index()
plt.pie(a.Y, labels = a.X)
plt.show()

a = full.groupby(['Y']).count().reset_index()
plt.pie(a.X, labels = a.Y)
plt.show() 

a = corrected.groupby(['X']).count().reset_index()
plt.pie(a.Y, labels = a.X)
plt.show()

a = corrected.groupby(['Y']).count().reset_index()
plt.pie(a.X, labels = a.Y)
plt.show()
corrected # hispanic 

In [ ]:
df = pd.read_csv('/content/drive/.shortcut-targets-by-id/1VhaW8PqOm10rxodZbXmRV2GHxBJq9Qxh/Data Open Championship 2022/2-data/preprocessed/fee_grad_pub.csv')
df.X = df.X.map(lambda x: x.rstrip('1234567890'))
df.Y = df.Y.map(lambda x: x.rstrip('1234567890'))
print(df.shape)
df = df.sort_values(by=['p_val']).dropna().drop(columns=['Unnamed: 0'])
full = df[df.p_val < 0.05]
corrected = df[smt.multipletests(df.p_val, method = 'holm')[0]]

a = full.groupby(['X']).count().reset_index()
plt.pie(a.Y, labels = a.X)
plt.show()

a = full.groupby(['Y']).count().reset_index()
plt.pie(a.X, labels = a.Y)
plt.show() 

a = corrected.groupby(['X']).count().reset_index()
plt.pie(a.Y, labels = a.X)
plt.show()

a = corrected.groupby(['Y']).count().reset_index()
plt.pie(a.X, labels = a.Y)
plt.show()
corrected

In [ ]:
df = pd.read_csv('/content/drive/.shortcut-targets-by-id/1VhaW8PqOm10rxodZbXmRV2GHxBJq9Qxh/Data Open Championship 2022/2-data/preprocessed/fee_grad_forpro.csv')
df.X = df.X.map(lambda x: x.rstrip('1234567890'))
df.Y = df.Y.map(lambda x: x.rstrip('1234567890'))
print(df.shape)
df = df.sort_values(by=['p_val']).dropna().drop(columns=['Unnamed: 0'])
full = df[df.p_val < 0.05]
corrected = df[smt.multipletests(df.p_val, method = 'holm')[0]]

a = full.groupby(['X']).count().reset_index()
plt.pie(a.Y, labels = a.X)
plt.show()

a = full.groupby(['Y']).count().reset_index()
plt.pie(a.X, labels = a.Y)
plt.show() 

a = corrected.groupby(['X']).count().reset_index()
plt.pie(a.Y, labels = a.X)
plt.show()

a = corrected.groupby(['Y']).count().reset_index()
plt.pie(a.X, labels = a.Y)
plt.show()
corrected

In [ ]:
df = pd.read_csv('/content/drive/.shortcut-targets-by-id/1VhaW8PqOm10rxodZbXmRV2GHxBJq9Qxh/Data Open Championship 2022/2-data/preprocessed/fee_grad_nonpro.csv')
df.X = df.X.map(lambda x: x.rstrip('1234567890'))
df.Y = df.Y.map(lambda x: x.rstrip('1234567890'))
print(df.shape)
df = df.sort_values(by=['p_val']).dropna().drop(columns=['Unnamed: 0'])
full = df[df.p_val < 0.05]
corrected = df[smt.multipletests(df.p_val, method = 'holm')[0]]

a = full.groupby(['X']).count().reset_index()
plt.pie(a.Y, labels = a.X)
plt.show()

a = full.groupby(['Y']).count().reset_index()
plt.pie(a.X, labels = a.Y)
plt.show() 

a = corrected.groupby(['X']).count().reset_index()
plt.pie(a.Y, labels = a.X)
plt.show()

a = corrected.groupby(['Y']).count().reset_index()
plt.pie(a.X, labels = a.Y)
plt.show()
corrected

In [ ]:
from statsmodels.formula.api import ols
import statsmodels.formula.api as smf

for i in range(6):
  X = trying[trying.columns[::27]].iloc[:,i]
  Y = grad[grad.columns[::105]].iloc[:,i]
  curr = pd.concat([X, Y], axis = 1).dropna()
  X = curr.iloc[:, 0]
  Y = curr.iloc[:, 1]
  X = sm.add_constant(X)
  model = sm.OLS(Y,X)
  print(model.fit().summary())
  fig = sm.graphics.plot_fit(model.fit(), list(X)[1]) #just for a few
  fig.tight_layout(pad=50.0)


In [ ]:
y = forproa[forproa.columns[forproa.columns.str.startswith("gradrate")]]

X = forproa[['finance_instruction-salaries and wages', 'time']]
X = sm.add_constant(X)
Y = forproa['gradrate_4yr_completer150_']
curr = pd.concat([X, Y], axis = 1).dropna()
curr = curr.rename(columns = {'finance_instruction-salaries and wages': 'cost1'})
model = smf.ols(formula='gradrate_4yr_completer150_ ~ cost1 + time', data=curr)
print(model.fit().summary())
fig = sm.graphics.plot_fit(model.fit(), 'cost1')
fig.tight_layout(pad=50.0)

In [ ]:
def get_all(main_df, name):
  aid = main_df[main_df.columns[main_df.columns.str.startswith("finaid")]]
  finance = main_df[main_df.columns[main_df.columns.str.startswith("finance")]]
  grad = main_df[main_df.columns[main_df.columns.str.startswith("gradrate")]]
  fee = main_df[main_df.columns[main_df.columns.str.startswith("fee")]]
  lst_df = []
  for i in range(22):
    lst_df.append(get_pvals(aid, grad, i, 27))
  pd.concat(lst_df).to_csv('/content/drive/.shortcut-targets-by-id/1VhaW8PqOm10rxodZbXmRV2GHxBJq9Qxh/Data Open Championship 2022/2-data/preprocessed/aid_grad_' + name + '.csv')
  for i in range(22):
    lst_df.append(get_pvals(finance, grad, i, 226))
  pd.concat(lst_df).to_csv('/content/drive/.shortcut-targets-by-id/1VhaW8PqOm10rxodZbXmRV2GHxBJq9Qxh/Data Open Championship 2022/2-data/preprocessed/finance_grad_' + name + '.csv')
  for i in range(22):
    lst_df.append(get_pvals(fee, grad, i, 25))
  pd.concat(lst_df).to_csv('/content/drive/.shortcut-targets-by-id/1VhaW8PqOm10rxodZbXmRV2GHxBJq9Qxh/Data Open Championship 2022/2-data/preprocessed/fee_grad_' + name + '.csv')

In [ ]:
def regress(main_df):
  aid = main_df[main_df.columns[main_df.columns.str.startswith("finaid")]]
  finance = main_df[main_df.columns[main_df.columns.str.startswith("finance")]]
  grad = main_df[main_df.columns[main_df.columns.str.startswith("gradrate")]]
  fee = main_df[main_df.columns[main_df.columns.str.startswith("fee")]]
  for i in range(k):
    curr_cost = df.iloc[:,k]

In [ ]:
forproa[forproa.columns[forproa.columns.str.startswith("fee")]]

In [ ]:
forproa[forproa.columns[forproa.columns.str.startswith("charac")]]